In [1]:
%%capture
!pip -q install langchain_experimental langchain_core
!pip -q install google-generativeai==0.3.1
!pip -q install google-ai-generativelanguage==0.4.0
!pip -q install langchain-google-genai
!pip -q install wikipedia
!pip -q install docarray
!pip -q install --upgrade protobuf google.protobuf

In [2]:
# Before proceeding with the next cells restart the kernel by clicking the refresh icon on the top toolbar.

In [3]:
import os
import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
from google.protobuf.empty_pb2 import Empty

key_name = !gcloud services api-keys list --filter="gemini-api-key" --format="value(name)"
key_name = key_name[0]

api_key = !gcloud services api-keys get-key-string $key_name --location="us-central1" --format="value(keyString)"
api_key = api_key[0]

os.environ["GOOGLE_API_KEY"] = api_key

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [4]:
models = [m for m in genai.list_models()]
models

[Model(name='models/chat-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 Chat (Legacy)',
       description='A legacy text-only model optimized for chat conversations',
       input_token_limit=4096,
       output_token_limit=1024,
       supported_generation_methods=['generateMessage', 'countMessageTokens'],
       temperature=0.25,
       max_temperature=None,
       top_p=0.95,
       top_k=40),
 Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 (Legacy)',
       description='A legacy model that understands text and generates text as an output',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       max_temperature=None,
       top_p=0.95,
       top_k=40),
 Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
      

# Using Gemini directly with Python SDK

In [5]:
# generate text
prompt = 'Who are you and what can you do?'

model = genai.GenerativeModel('gemini-pro')

response = model.generate_content(prompt)

Markdown(response.candidates[0].content.parts[0].text)

I am Gemini, a multi-modal AI model, developed by Google. I am designed to provide information and assist users with a wide range of topics and tasks. Here are some of my capabilities:

**Information Retrieval and Generation:**
* Answer questions on various topics, including science, history, current events, and more.
* Generate text in different styles and tones, such as summaries, articles, stories, and poems.
* Translate text between over 100 languages.

**Conversational Skills:**
* Engage in natural language conversations and understand the context and intent of user queries.
* Provide personalized responses based on the user's preferences and history.
* Generate creative responses, such as jokes, puns, and riddles.

**Task Assistance:**
* Set alarms, timers, and reminders.
* Perform mathematical calculations and unit conversions.
* Provide weather forecasts and news updates.
* Help with scheduling appointments and making reservations.

**Knowledge and Understanding:**
* Possess a vast knowledge base across a wide range of domains, including:
    * Science and technology
    * Arts and culture
    * History and geography
    * Business and finance
* Continuously learn and update my knowledge through ongoing training.

**Additional Features:**
* Access and process information from the internet and various databases.
* Integrate with external applications and services through APIs.
* Generate images, music, and other creative content.

Please note that while I am capable of performing many tasks and providing information, I am still under development and my abilities are constantly evolving. If you have any specific questions or requests, feel free to ask, and I will do my best to assist you.

# Using Gemini with LangChain

## Basic LLM Chain

In [6]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.7)


result = llm.invoke("What is a LLM?")

Markdown(result.content)

LLM stands for Large Language Model. LLMs are a type of artificial intelligence (AI) that are trained on massive datasets of text and code. They are able to understand and generate human-like text, translate languages, write different kinds of creative content, and even write different kinds of code.

LLMs are used in a wide variety of applications, including:

* **Natural language processing (NLP)**: LLMs can be used to perform a variety of NLP tasks, such as text classification, sentiment analysis, machine translation, and question answering.
* **Machine learning (ML)**: LLMs can be used to train ML models on text data. This can be useful for tasks such as image classification, speech recognition, and recommendation systems.
* **Creative content generation**: LLMs can be used to generate creative content, such as stories, poems, and songs.
* **Code generation**: LLMs can be used to generate code in a variety of programming languages. This can be useful for tasks such as software development and bug fixing.

LLMs are still under development, but they have the potential to revolutionize a wide range of industries. They are already being used to create new products and services, and they are likely to play an even bigger role in our lives in the years to come.

In [7]:
for chunk in llm.stream("Write a haiku about LLMs."):
    print(chunk.content)
    print("---")

Words flow like a stream
AI's mimicry, it gleams

---
Language's new dream
---


## Basic Multi Chain

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser


model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

In [9]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)

output_parser = StrOutputParser()

In [10]:
chain = prompt | model | output_parser

In [11]:
chain.invoke({"topic": "machine learning"})

'Why did the machine learning algorithm get lost in the forest?\n\nBecause it overfit the training data.'

## A more complicated Chain - RAG

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [13]:
model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.7)

In [14]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [15]:
from langchain.document_loaders import WikipediaLoader

# use Wikipedia loader to create some docs to use..
docs = WikipediaLoader(query="Machine Learning", load_max_docs=10).load()
docs += WikipediaLoader(query="Deep Learning", load_max_docs=10).load() 
docs += WikipediaLoader(query="Neural Networks", load_max_docs=10).load() 

# Take a look at a single document
docs[0]

Document(metadata={'title': 'Machine learning', 'summary': "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions. Recently, artificial neural networks have been able to surpass many previous approaches in performance.\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through u

In [16]:
vectorstore = DocArrayInMemorySearch.from_documents(
    docs,
    embedding=embeddings # passing in the model to embed documents..
)

retriever = vectorstore.as_retriever()

2024-07-15 11:21:35.923191: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 11:21:36.245646: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-15 11:21:36.249293: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-15 11:21:36.754615: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 11:21:41.626153: W tensorflow/compiler/tf

In [17]:
retriever.get_relevant_documents("what is machine learning?")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(metadata={'title': 'Machine learning', 'summary': "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions. Recently, artificial neural networks have been able to surpass many previous approaches in performance.\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through 

In [18]:
retriever.get_relevant_documents("what is gemini pro?")

[Document(metadata={'title': 'Graph neural network', 'summary': 'A graph neural network (GNN) belongs to a class of artificial neural networks for processing data that can be represented as graphs.\n\nIn the more general subject of "geometric deep learning", certain existing neural network architectures can be interpreted as GNNs operating on suitably defined graphs. A convolutional neural network layer, in the context of computer vision, can be seen as a GNN applied to graphs whose nodes are pixels and only adjacent pixels are connected by edges in the graph. A transformer layer, in natural language processing, can be seen as a GNN applied to complete graphs whose nodes are words or tokens in a passage of natural language text.\nThe key design element of GNNs is the use of pairwise message passing, such that graph nodes iteratively update their representations by exchanging information with their neighbors. Since their inception, several different GNN architectures have been proposed,

In [19]:
template = """Answer the question a a full sentence, based only on the following context:
{context}

Return you answer in three back ticks

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [20]:
from langchain.schema.runnable import RunnableMap

In [21]:
retriever.get_relevant_documents("What is a graident boosted tree?")

[Document(metadata={'title': 'Boosting (machine learning)', 'summary': 'In machine learning, boosting is an ensemble meta-algorithm for primarily reducing bias, variance. It is used in supervised learning and a family of machine learning algorithms that convert weak learners to strong ones.\nThe concept of boosting is based on the question posed by Kearns and Valiant (1988, 1989): "Can a set of weak learners create a single strong learner?" A weak learner is defined to be a classifier that is only slightly correlated with the true classification (it can label examples better than random guessing). In contrast, a strong learner is a classifier that is arbitrarily well-correlated with the true classification.\nRobert Schapire answered the question posed by Kearns and Valiant in the affirmative in a paper published in 1990.This has had significant ramifications in machine learning and statistics, most notably leading to the development of boosting. \nWhen first introduced, the hypothesis 

In [22]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [23]:
chain.invoke({"question": "What is machine learning?"})

'```Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.```'

In [24]:
chain.invoke({"question": "When was the transformer invented?"})

'```\n2017\n```'